In [52]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import f1_score
#from fastai.conv_learner import *
#from fastai.dataset import *
import scipy.optimize as opt
from scipy.special import logit

In [18]:
from fastai.basic_train import

In [8]:
LABELS = '../../Human_Protein_Atlas/input/train.csv'
SAMPLE = '../../Human_Protein_Atlas/input/sample_submission.csv'

In [6]:
name_label_dict = {
0:  'Nucleoplasm',
1:  'Nuclear membrane',
2:  'Nucleoli',   
3:  'Nucleoli fibrillar center',
4:  'Nuclear speckles',
5:  'Nuclear bodies',
6:  'Endoplasmic reticulum',   
7:  'Golgi apparatus',
8:  'Peroxisomes',
9:  'Endosomes',
10:  'Lysosomes',
11:  'Intermediate filaments',
12:  'Actin filaments',
13:  'Focal adhesion sites',   
14:  'Microtubules',
15:  'Microtubule ends',  
16:  'Cytokinetic bridge',   
17:  'Mitotic spindle',
18:  'Microtubule organizing center',  
19:  'Centrosome',
20:  'Lipid droplets',
21:  'Plasma membrane',   
22:  'Cell junctions', 
23:  'Mitochondria',
24:  'Aggresome',
25:  'Cytosol',
26:  'Cytoplasmic bodies',   
27:  'Rods & rings' }

In [9]:
preds1 = np.load('../../Human_Protein_Atlas/preds/resnet34_460_1.npy')
preds2 = np.load('../../Human_Protein_Atlas/preds/resnet34_460_2.npy')
preds3 = np.load('../../Human_Protein_Atlas/preds/resnet34_460_3.npy')
preds4 = np.load('../../Human_Protein_Atlas/preds/resnet34_460_4.npy')
preds5 = np.load('../../Human_Protein_Atlas/preds/resnet34_460_5.npy')
preds6 = np.load('../../Human_Protein_Atlas/preds/resnet34_460_6.npy')
preds7 = np.load('../../Human_Protein_Atlas/preds/resnet34_460_7.npy')
preds8 = np.load('../../Human_Protein_Atlas/preds/resnet34_460_8.npy')
preds9 = np.load('../../Human_Protein_Atlas/preds/resnet34_460_9.npy')
preds10 = np.load('../../Human_Protein_Atlas/preds/resnet34_460_10.npy')

In [3]:
preds1.shape

(20, 11702, 28)

In [10]:
preds = 0.1*(preds1+preds2+preds3+preds4+preds5+preds6+preds7+preds8+preds9+preds10)

In [11]:
def sigmoid_np(x):
    return 1.0/(1.0 + np.exp(-x))

preds = np.stack(preds, axis=-1)
preds = sigmoid_np(preds)
pred = preds.max(axis=-1)

In [38]:
preds1 = np.stack(preds1, axis=-1)
preds1 = sigmoid_np(preds1)
pred1 = preds1.max(axis=-1)

In [37]:
pred[0]

array([5.2348232e-01, 2.0142082e-02, 2.2491321e-01, 8.2294092e-02,
       4.2780086e-02, 9.1223091e-02, 5.9451494e-02, 4.2408485e-02,
       2.5271308e-03, 5.6514208e-04, 3.0834426e-04, 4.2344563e-02,
       5.0072327e-02, 3.3695813e-02, 5.7966933e-02, 6.1218496e-03,
       1.0156636e-01, 2.8347801e-02, 3.6339227e-02, 7.1931377e-02,
       7.2064814e-03, 1.6035599e-01, 4.6876181e-02, 9.2335522e-02,
       6.6227796e-03, 3.5865808e-01, 2.5644226e-02, 7.6769362e-04],
      dtype=float32)

In [12]:
pred.shape

(11702, 28)

In [13]:
th_t = np.array([0.565,0.39,0.55,0.345,0.33,0.39,0.33,0.45,0.38,0.39,
               0.34,0.42,0.31,0.38,0.49,0.50,0.38,0.43,0.46,0.40,
               0.39,0.505,0.37,0.47,0.41,0.545,0.32,0.1])

In [32]:
learner_data_test_ds_fnames = np.load('../../Human_Protein_Atlas/models/learner.data.test_ds.fnames.npy').tolist()

In [31]:
1

1

In [33]:
def save_pred(pred, th=0.5, fname='protein_classification.csv'):
    pred_list = []
    for line in pred:
        s = ' '.join(list([str(i) for i in np.nonzero(line>th)[0]]))
        pred_list.append(s)
        
    sample_df = pd.read_csv(SAMPLE)
    sample_list = list(sample_df.Id)
    pred_dic = dict((key, value) for (key, value) 
                in zip(learner_data_test_ds_fnames,pred_list))
    pred_list_cor = [pred_dic[id] for id in sample_list]
    df = pd.DataFrame({'Id':sample_list,'Predicted':pred_list_cor})
    df.to_csv(fname, header=True, index=False)

In [39]:
print('Fractions: ',(pred1 > th_t).mean(axis=0))
save_pred(pred1,th_t)

Fractions:  [4.54024953e-01 5.05041873e-02 1.07332080e-01 6.08443001e-02
 6.49461630e-02 1.26815929e-01 5.83660913e-02 1.12886686e-01
 1.79456503e-03 1.02546573e-03 8.54554777e-04 5.83660913e-02
 4.50350367e-02 1.94838489e-02 4.46077594e-02 1.70910955e-04
 2.76875748e-02 1.48692531e-02 3.54640232e-02 6.62279952e-02
 7.00734917e-03 1.81336524e-01 3.23021706e-02 1.21859511e-01
 1.67492736e-02 3.16356178e-01 2.68330200e-02 1.88002051e-03]


In [40]:
preds2 = np.stack(preds2, axis=-1)
preds2 = sigmoid_np(preds2)
pred2 = preds2.max(axis=-1)
print('Fractions: ',(pred2 > th_t).mean(axis=0))
save_pred(pred2,th_t)

Fractions:  [4.53597676e-01 4.64877799e-02 1.10237566e-01 5.69988036e-02
 6.11861220e-02 1.11006666e-01 6.03315673e-02 1.04682960e-01
 2.30729790e-03 1.62365408e-03 1.62365408e-03 5.48624167e-02
 4.63168689e-02 2.04238592e-02 4.35822936e-02 2.56366433e-04
 2.17911468e-02 1.66638182e-02 3.62331225e-02 8.11827038e-02
 6.32370535e-03 1.77918305e-01 3.18748932e-02 1.30832336e-01
 1.46128867e-02 3.30541788e-01 2.82857631e-02 4.52914032e-03]


In [41]:
print('Fractions: ',(0.5*pred2 + 0.5*pred1 > th_t).mean(axis=0))
save_pred(0.5*pred2 + 0.5*pred1,th_t)

Fractions:  [3.71218595e-01 5.28969407e-02 4.46077594e-02 5.18714750e-02
 6.50316185e-02 9.63937788e-02 4.72568792e-02 8.65663989e-02
 8.54554777e-05 3.41821911e-04 5.12732866e-04 4.69150573e-02
 4.06768074e-02 1.58947189e-02 1.99965818e-02 0.00000000e+00
 1.38437874e-02 3.67458554e-03 6.40916083e-03 4.28986498e-02
 5.12732866e-04 8.89591523e-02 1.88856606e-02 8.47718339e-02
 1.05110238e-02 2.25089728e-01 1.62365408e-02 8.54554777e-04]


In [42]:
preds3 = np.stack(preds3, axis=-1)
preds3 = sigmoid_np(preds3)
pred3 = preds3.max(axis=-1)
print('Fractions: ',(pred3 > th_t).mean(axis=0))
save_pred(pred3,th_t)

Fractions:  [0.45419586 0.05212784 0.09647923 0.06135703 0.06964621 0.12621774
 0.04118954 0.10528115 0.00170911 0.00205093 0.00111092 0.04588959
 0.04409503 0.01777474 0.04828234 0.         0.02247479 0.0147838
 0.03657494 0.06827893 0.00888737 0.1762092  0.03366946 0.10399932
 0.01410015 0.34694924 0.03349855 0.00410186]


In [43]:
preds4 = np.stack(preds4, axis=-1)
preds4 = sigmoid_np(preds4)
pred4 = preds4.max(axis=-1)
print('Fractions: ',(pred4 > th_t).mean(axis=0))
save_pred(pred4,th_t)

Fractions:  [0.46880875 0.05007691 0.11502307 0.05836609 0.06323705 0.1028884
 0.0529824  0.12587592 0.00205093 0.00179457 0.00128183 0.0529824
 0.04187318 0.01572381 0.04076226 0.         0.02666211 0.01504016
 0.04563323 0.07272261 0.00598188 0.16971458 0.03221672 0.11399761
 0.01392924 0.36874039 0.02914032 0.00145274]


In [44]:
print('Fractions: ',(0.25*pred2 + 0.25*pred1 + 0.25*pred3 + 0.25*pred4 > th_t).mean(axis=0))
save_pred(0.25*pred2 + 0.25*pred1 + 0.25*pred3 + 0.25*pred4,th_t)

Fractions:  [3.49598359e-01 7.60553751e-03 1.02546573e-02 1.06819347e-02
 1.84583832e-02 3.16185267e-02 9.91283541e-03 2.38420783e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.46915057e-03
 1.00837464e-02 5.98188344e-04 2.30729790e-03 0.00000000e+00
 1.36728764e-03 0.00000000e+00 8.54554777e-05 7.43462656e-03
 0.00000000e+00 3.21312596e-02 1.88002051e-03 1.89711160e-02
 4.27277388e-04 1.59459921e-01 1.11092121e-03 3.41821911e-04]


In [46]:
print('Fractions: ',(pred > th_t**1.2).mean(axis=0))
save_pred(pred,th_t**1.2)

Fractions:  [2.90804991e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 2.56366433e-04 0.00000000e+00 2.56366433e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.28183217e-03 0.00000000e+00 1.70910955e-04
 0.00000000e+00 4.52059477e-02 0.00000000e+00 0.00000000e+00]


In [55]:
print('Fractions: ',(pred > th_t).mean(axis=0))
save_pred(pred,th_t)

Fractions:  [1.63732695e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.54554777e-05
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.70910955e-04 0.00000000e+00 8.54554777e-05
 0.00000000e+00 1.04255683e-02 0.00000000e+00 0.00000000e+00]


In [50]:
print('Fractions: ',(pred > th_t**1.4).mean(axis=0))
save_pred(pred,th_t**1.4)

Fractions:  [4.23260981e-01 0.00000000e+00 2.56366433e-04 0.00000000e+00
 3.41821911e-04 2.73457529e-03 1.70910955e-04 8.54554777e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 8.54554777e-05 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 3.84549650e-03 0.00000000e+00 5.12732866e-04
 0.00000000e+00 1.26730473e-01 0.00000000e+00 0.00000000e+00]


In [51]:
print('Fractions: ',(pred > th_t**1.45).mean(axis=0))
save_pred(pred,th_t**1.45)

Fractions:  [4.57870449e-01 0.00000000e+00 2.56366433e-04 8.54554777e-05
 4.27277388e-04 3.16185267e-03 5.12732866e-04 1.19637669e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 8.54554777e-05 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 5.81097248e-03 0.00000000e+00 5.98188344e-04
 0.00000000e+00 1.54418048e-01 0.00000000e+00 0.00000000e+00]


In [54]:
1-logit(th_t)

array([0.7385203 , 1.44731222, 0.7993293 , 1.64109082, 1.70818506,
       1.44731222, 1.70818506, 1.2006707 , 1.48954823, 1.44731222,
       1.66329422, 1.32277339, 1.8001193 , 1.48954823, 1.04000533,
       1.        , 1.48954823, 1.28185115, 1.16034265, 1.40546511,
       1.44731222, 0.97999933, 1.53221681, 1.12014431, 1.36396538,
       0.81951162, 1.7537718 , 3.19722458])

In [ ]:
print('Fractions: ',(pred > th_t**1.45).mean(axis=0))
save_pred(pred,th_t**1.45)